# 1. Parameters

In [1]:
cases_dir = 'cases/unset'
metadata_file = 'input/metadata-subsample-pangolin.tsv'

In [2]:
from pathlib import Path
import imp
fp, pathname, description = imp.find_module('gdi_benchmark', ['../../lib'])
gdi_benchmark = imp.load_module('gdi_benchmark', fp, pathname, description)

cases_dir_path = Path(cases_dir)

case_name = str(cases_dir_path.name)
index_path = cases_dir_path / 'index'

output_api_path = cases_dir_path / 'query-api.tsv'
output_cli_path = cases_dir_path / 'query-cli.tsv'

# 2. Benchmark command-line

In [3]:
import pandas as pd
import genomics_data_index.api as gdi

def benchmark_cli_index(name: str, index_path: Path) -> pd.DataFrame:
    benchmark_commands = {
        'query hasa':               f'gdi --project-dir {index_path} --ncores 1 query "hasa:hgvs_gn:NC_045512.2:S:p.D614G"',
        'query isa':                f'gdi --project-dir {index_path} --ncores 1 query "isa:Switzerland/100835/2020"',
        'query isin':               f'gdi --project-dir {index_path} --ncores 1 query --reference-name NC_045512 "isin_5_substitutions:Switzerland/100835/2020"',
        'query --summary':          f'gdi --project-dir {index_path} --ncores 1 query --summarize',
        'query --features-summary': f'gdi --project-dir {index_path} --ncores 1 query --features-summary',
        'list samples':             f'gdi --project-dir {index_path} --ncores 1 list samples',
    }
    
    db = gdi.GenomicsDataIndex.connect(index_path)
    number_samples = db.count_samples()
    number_features_no_unknown = db.count_mutations(reference_genome='NC_045512', include_unknown=False)
    number_features_all = db.count_mutations(reference_genome='NC_045512', include_unknown=True)
    iterations = 10
    
    benchmarker = gdi_benchmark.QueryBenchmarkHandler()
    return benchmarker.benchmark_cli(name=name, kind_commands=benchmark_commands, number_samples=number_samples,
                                    number_features_no_unknown=number_features_no_unknown, number_features_all=number_features_all,
                                     iterations=iterations)

In [4]:
cli_df = benchmark_cli_index(name=case_name, index_path=index_path)
cli_df.head(3)

,Name,Kind,Iteration,Number samples,Number features (no unknown),Number features (all),Runtime,Memory (max),Mmemory (max/process)
0,unset,query hasa,1,10,79,2633,2.61,220798976.0,220798976.0
0,unset,query hasa,2,10,79,2633,2.95,221044736.0,221044736.0
0,unset,query hasa,3,10,79,2633,2.78,220921856.0,220921856.0


In [5]:
cli_df.to_csv(output_cli_path, sep='\t', index=False)

# 3. Test query API

## 3.1. Load (example) metadata

The simulated data is based off of real sample names and a real tree. So I can load up real metadata and attach it to a query (though the mutations and reference genome are all simulated).

In [6]:
import pandas as pd

metadata_df = pd.read_csv(metadata_file, sep='\t')
metadata_df.head(2)

,strain,virus,gisaid_epi_isl,genbank_accession,sra_accession,date,region,country,division,location,...,ambiguity_score,scorpio_call,scorpio_support,scorpio_conflict,version,pangolin_version,pangoLEARN_version,pango_version,status,note
0,England/QEUH-9F48A4/2020,ncov,EPI_ISL_588307,OA992707,ERR4688590,2020-09-25,Europe,United Kingdom,England,NaN,...,NaN,NaN,NaN,NaN,PANGO-v1.2.66,3.1.11,2021-08-24,v1.2.66,passed_qc,Assigned from designation hash.
1,USA/UT-CDC-QDX26118077/2021,ncov,EPI_ISL_2873924,MZ534972,NaN,2021-06-22,North America,USA,Utah,NaN,...,1.0,Delta (B.1.617.2-like),1.0,0.0,PLEARN-v1.2.66,3.1.11,2021-08-24,v1.2.66,passed_qc,scorpio call: Alt alleles 13; Ref alleles 0; A...


## 3.2. Define benchmark cases

In [10]:
from typing import List
import genomics_data_index.api as gdi

def benchmark_api_index(name: str, index_path: Path) -> pd.DataFrame:
    db = gdi.GenomicsDataIndex.connect(index_path)
    q_no_join = db.samples_query(reference_name='NC_045512', universe='mutations')
    q_join = db.samples_query(reference_name='NC_045512', universe='mutations').join(metadata_df, sample_names_column='strain')

    q = q_join.hasa('hgvs_gn:NC_045512.2:S:p.D614G')
    r = q_join.hasa('hgvs_gn:NC_045512.2:N:p.R203K')
    
    number_samples = db.count_samples()
    number_features_no_unknown = db.count_mutations(reference_genome='NC_045512', include_unknown=False)
    number_features_all = db.count_mutations(reference_genome='NC_045512', include_unknown=True)
    repeat = 10

    benchmark_cases = {
        'db.samples_query':      lambda: db.samples_query(reference_name='NC_045512', universe='mutations'),
        'q.join':                lambda: q_no_join.join(metadata_df, sample_names_column='strain'),
        'q.features_summary':    lambda: q_join.features_summary(),
        'q.features_comparison': lambda: q_join.features_comparison(sample_categories='lineage', categories_kind='dataframe', kind='mutations', unit='proportion'),
        'q.hasa':                lambda: q_join.hasa("hgvs_gn:NC_045512.2:N:p.R203K"),
        'q.isa':                 lambda: q_join.isa("Switzerland/100835/2020"),
        'q.isin (distance)':     lambda: q_join.isin("Switzerland/100835/2020", kind='distance', distance=5, units='substitutions'),
        'q.isin (mrca)':         lambda: q_join.isin(["Switzerland/100835/2020", "Switzerland/180011/2020"], kind='mrca'),
        'q AND r':               lambda: q & r,
        'q.toframe':             lambda: q_join.toframe(),
        'q.summary':             lambda: q_join.summary(),
    }

    benchmarker = gdi_benchmark.QueryBenchmarkHandler()
    return benchmarker.benchmark_api(name=name, kind_functions=benchmark_cases,
                                       number_samples=number_samples,
                                       number_features_no_unknown=number_features_no_unknown,
                                       number_features_all=number_features_all,
                                       repeat=repeat)

## 3.3. Benchmark reads index

In [11]:
api_df = benchmark_api_index(name=case_name, index_path=index_path)
api_df.head(5)

Attempting to set global database_path_translator=<genomics_data_index.storage.model.db.DatabasePathTranslator.DatabasePathTranslator object at 0x7fcd29b00070> but it is already set


,Name,Kind,Number samples,Number features (no unknown),Number features (all),Number executions,Iteration,Time
0,unset,db.samples_query,10,79,2633,500,1,0.000907
1,unset,db.samples_query,10,79,2633,500,2,0.000907
2,unset,db.samples_query,10,79,2633,500,3,0.000905
3,unset,db.samples_query,10,79,2633,500,4,0.000910
4,unset,db.samples_query,10,79,2633,500,5,0.000906


In [12]:
api_df.to_csv(output_api_path, sep='\t', index=False)